In [1]:
import os
import re
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Preprocessing

In [2]:
cwd = os.getcwd()
main_table = pd.read_csv(os.path.join(cwd, '../MainTable.csv'))
code = pd.read_csv(os.path.join(cwd, '../LinkTables/CodeStates.csv'))
questions = pd.read_excel(os.path.join(cwd, '../LinkTables/questions.xlsx'))

In [3]:
# change the time to the same zone time
main_table.loc[main_table['ServerTimezone'] == '0', 'ServerTimestamp'] = pd.to_datetime(main_table.loc[main_table['ServerTimezone'] == '0', 'ServerTimestamp']).dt.tz_localize('US/Eastern')

main_table.loc[main_table['ServerTimezone'] == 'UTC', 'ServerTimestamp'] = pd.to_datetime(main_table.loc[main_table['ServerTimezone'] == 'UTC', 'ServerTimestamp']).dt.tz_localize('UTC')

main_table['ServerTimestamp'] = pd.to_datetime(main_table['ServerTimestamp'], utc=True)

In [4]:
# Each student+question row in table has 2-3 rows for the compilition results. lets remove it:
main_table = main_table[main_table['Score'].notnull()]

In [5]:
subtable = main_table[['SubjectID', 'ServerTimestamp', 'CourseSectionID', 'AssignmentID', 'ProblemID', 'CodeStateID', 'Score']]
subtable = subtable.sort_values(by='ServerTimestamp')

In [6]:
df = subtable.groupby(['SubjectID', 'CourseSectionID', 'AssignmentID', 'ProblemID']).apply(lambda x: pd.Series({
    'start_time': x['ServerTimestamp'].iloc[0],
    'code_state_ID': x['CodeStateID'].tolist(),
    'score': x['Score'].tolist(),
    'server_timestamp': x['ServerTimestamp'].tolist(),
})).reset_index()

df['num_snapshots'] = df['code_state_ID'].apply(lambda x: len(x))
df['time_to_solve'] = df['server_timestamp'].apply(lambda x: (pd.to_datetime(x[-1]) - pd.to_datetime(x[0])).seconds)
df['time_between_snapshots'] = df['server_timestamp'].apply(lambda x: [(pd.to_datetime(x[i]) - pd.to_datetime(x[i-1])).seconds for i in range(1, len(x))])
df['struggling'] = df['score'].apply(lambda x: 0 if x[-1] == 1 else 1)

/tmp/ipykernel_3421728/2381715478.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = subtable.groupby(['SubjectID', 'CourseSectionID', 'AssignmentID', 'ProblemID']).apply(lambda x: pd.Series({


In [7]:
snapshots = []
for i in df['code_state_ID']:
    snapshots_student = []
    for j in i:
        snapshots_student.append(code[code['CodeStateID'] == j]['Code'].values[0])
    snapshots.append(snapshots_student)

df['source_code'] = snapshots

In [8]:
df = df.rename(columns={'SubjectID': 'student_id'})
df = df.rename(columns={'CourseSectionID': 'course_id'})
df = df.rename(columns={'AssignmentID': 'assignment_id'})
df = df.rename(columns={'ProblemID': 'problem_id'})
df['max_score'] = df['score'].apply(lambda x: max(x) * 100)
df = df.drop(columns=['code_state_ID'])
df = df.sort_values(by='start_time')

In [9]:
df_basic_model = df[['student_id', 'course_id', 'assignment_id', 'problem_id', 'start_time', 'score', 'source_code', 'struggling']]

# Add Coding Task text & Students' codes

In [10]:
q = []
for i, row in df_basic_model.iterrows():
    q.append(questions[(questions['AssignmentID'] == row['assignment_id']) & (questions['ProblemID'] == row['problem_id'])]['Requirement'].values[0])
df_basic_model['question'] = q

/tmp/ipykernel_3421728/4242415766.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_basic_model['question'] = q


In [11]:
df_sorted = df_basic_model.sort_values(by=['student_id', 'start_time'])
df_sorted['row_number'] = df_sorted.groupby('student_id').cumcount() + 1
df_sorted['question_num'] = df_sorted['assignment_id'].astype(str) + '_' + df_sorted['problem_id'].astype(str)

In [12]:
prev_code = df_sorted['source_code'].tolist()
prev_code.insert(0, [])
prev_q = df_sorted['question'].tolist()
prev_q.insert(0, [])
prev_num_q = df_sorted['question_num'].tolist()
prev_num_q.insert(0, [])
prev_score = df_sorted['score'].tolist()
prev_score.insert(0, [])

In [13]:
df_sorted['prev_code'] = prev_code[:-1] # last question of the last student
df_sorted['prev_question'] = prev_q[:-1]
df_sorted['prev_question_num'] = prev_num_q[:-1]
df_sorted['score'] = prev_score[:-1]

df_sorted = df_sorted[df_sorted['row_number'] != 1]

In [14]:
prev_code = []
prev_question = []
prev_q_num = []
prev_score = []
for i, row in df_sorted.iterrows():
    if row['row_number'] == 2:
        prev_code.append([row['prev_code']])
        prev_question.append([row['prev_question']])
        prev_q_num.append([row['prev_question_num']])
        prev_score.append([row['score']])
    else:
        prev_code.append(prev_code[-1] + [row['prev_code']])
        prev_question.append(prev_question[-1] + [row['prev_question']])
        prev_q_num.append(prev_q_num[-1] + [row['prev_question_num']])
        prev_score.append(prev_score[-1] + [row['score']])

In [15]:
df_sorted['prev_code'] = prev_code
df_sorted['prev_question'] = prev_question
df_sorted['prev_question_num'] = prev_q_num
df_sorted['score'] = prev_score

In [16]:
df_basic_model = df_sorted[['student_id', 'course_id', 'assignment_id', 'problem_id', 'prev_code', 'prev_question', 'question', 'struggling']]

In [17]:
df_basic_model.to_csv(os.path.join(cwd, '..\basic_model.csv'), index=False)

# Taxonomy

In [27]:
q_with_compu = questions.copy()
q_with_compu['IfKnowledge'] = q_with_compu[['If/Else', 'NestedIf']].max(axis=1)
q_with_compu['StringKnowledge'] = q_with_compu[['StringConcat', 'StringIndex', 'StringFormat']].max(axis=1)
q_with_compu['LoopsKnowledge'] = q_with_compu[['While', 'For', 'NestedFor']].max(axis=1)
q_with_compu['Math+-*/%'] = q_with_compu[['Math+-*/', 'Math%']].max(axis=1)
q_with_compu['LogicOperators'] = q_with_compu[['LogicAndNotOr', 'LogicCompareNum']].max(axis=1)

q_with_compu = q_with_compu.drop(columns=['If/Else', 'NestedIf', 'StringConcat', 'StringIndex', 'StringFormat',
                      'While', 'For', 'NestedFor', 'Math+-*/', 'Math%', 'LogicAndNotOr', 'LogicCompareNum', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'])
q_with_compu.fillna(0, inplace=True)
questions.fillna(0, inplace=True)

In [28]:
questions.columns

Index(['AssignmentID', 'ProblemID', 'Requirement', 'If/Else', 'NestedIf',
       'While', 'For', 'NestedFor', 'Math+-*/', 'Math%', 'LogicAndNotOr',
       'LogicCompareNum', 'LogicBoolean', 'StringFormat', 'StringConcat',
       'StringIndex', 'StringLen', 'StringEqual', 'CharEqual', 'ArrayIndex',
       'DefFunction', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'],
      dtype='object')

In [29]:
df_taxonomy = pd.merge(df_sorted, q_with_compu, left_on=['assignment_id', 'problem_id'], right_on=['AssignmentID', 'ProblemID'], how='outer')

In [30]:
df_taxonomy =df_taxonomy[['student_id', 'course_id', 'assignment_id', 'problem_id', 'prev_code', 'prev_question_num',
                          'prev_question', 'score', 'question', 'struggling', 'If/Else', 'NestedIf',
       'While', 'For', 'NestedFor', 'Math+-*/', 'Math%', 'LogicAndNotOr', 'LogicCompareNum', 'LogicBoolean', 'StringFormat', 'StringConcat',
       'StringIndex', 'StringLen', 'StringEqual', 'CharEqual', 'ArrayIndex','DefFunction']]
df_taxonomy['question_num'] = df_taxonomy['assignment_id'].astype(str) + '_' + df_taxonomy['problem_id'].astype(str)

In [31]:
df_taxonomy = df_taxonomy.drop(columns=['question_num', 'prev_question_num'])

In [32]:
df_taxonomy.to_csv(os.path.join(cwd, '../df_taxonomy_all.csv'), index=False)

# Sort By Similarity

In [ ]:
df_similarity = df_taxonomy.copy()

In [22]:
# Caculate similarity of questions
questions['Key'] = questions['AssignmentID'].astype('float64').astype(str) + '_' + questions['ProblemID'].astype(str)
questions.set_index('Key', inplace=True)
feature_columns = ['If/Else', 'NestedIf',
       'While', 'For', 'NestedFor', 'Math+-*/', 'Math%', 'LogicAndNotOr',
       'LogicCompareNum', 'LogicBoolean', 'StringFormat', 'StringConcat',
       'StringIndex', 'StringLen', 'StringEqual', 'CharEqual', 'ArrayIndex',
       'DefFunction']
similarity_matrix = cosine_similarity(questions[feature_columns])
similarity_df = pd.DataFrame(similarity_matrix, index=questions.index, columns=questions.index)

In [33]:
def sort_by_similarity(row):
    questions_num = row['question_num']
    similar_keys = similarity_df.loc[questions_num].sort_values(ascending=False)

    # Sort the lists based on the similarity order
    sorted_tuple = sorted(zip(row['prev_code'], row['prev_question_num'], row['prev_question'], row['score']), key=lambda pair: similar_keys.get(pair[1], float('inf')), reverse=True)
    code, q_num, q, score = zip(*sorted_tuple) 
    
    return pd.Series([list(code), list(q_num), list(q), list(score)])

In [34]:
df_taxonomy[['prev_code', 'prev_question_num', 'prev_question', 'score']] = df_taxonomy.apply(sort_by_similarity, axis=1)

# Meta Data